# Loading Libraries

In [ ]:
import re 
import warnings 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
import random 
import math
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils as torch_utils
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence, unpack_sequence, PackedSequence, pad_sequence
from tqdm import tqdm_notebook
from sklearn.preprocessing import MaxAbsScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer, MinMaxScaler
import torch.nn.functional as F
import d2l
import time
import traceback
import fastprogress
from torchmetrics.classification import BinaryAccuracy, Accuracy 
import torch.nn.init as init
import torch.optim.lr_scheduler as lr_scheduler
from itertools import repeat
from torch.optim.lr_scheduler import ReduceLROnPlateau
import datetime
import h5py
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import importlib
from scipy import stats
from time import perf_counter, sleep

# Functions and models

In [ ]:
class LinearReadout(nn.Module):
    """
    A simple linear readout layer for mapping input features to a single output.

    Args:
        input_size (int): The size of the input features.

    Attributes:
        linear (nn.Linear): A fully connected linear layer that maps the input features to a single output.

    Methods:
        forward(x):
            Applies the linear transformation to the input and squeezes the output along the third dimension.

    Forward Args:
        x (torch.Tensor): A tensor of shape (batch_size, seq_len, input_size) representing the input features.

    Returns:
        torch.Tensor: A tensor of shape (batch_size, seq_len) representing the output after applying the linear transformation.
    """
    def __init__(self, input_size):
        super().__init__()
        self.linear = nn.Linear(input_size, 1)
        
    def forward(self, x):
        x = self.linear(x)
        return x.squeeze(dim=2)

In [ ]:
class NonlinearDecoder(nn.Module):
    """
    A neural network module that decodes hidden representations into output predictions using a linear layer.

    Args:
        hidden_size (int): The size of the input hidden representations.
        output_size (int): The size of the output predictions.
        dropout (float): Dropout probability for regularization. Default is 0.0 (no dropout).
        device (torch.device, optional): The device to which the module is assigned. Default is None.

    Attributes:
        linear (nn.Linear): A fully connected linear layer that maps hidden representations to output predictions.
        dropout (nn.Dropout): A dropout layer for regularization.

    Methods:
        forward(factors):
            Processes a batch of hidden representations through the linear layer and returns the output predictions.

    Forward Args:
        factors (list of torch.Tensor): A list of tensors, where each tensor represents a sequence of hidden representations.

    Returns:
        list of torch.Tensor: A list of tensors, where each tensor represents the decoded output predictions for the corresponding input sequence.

    Notes:
        - The input `factors` is a list of tensors with varying sequence lengths.
        - The method concatenates all sequences, processes them through the linear layer, and splits them back into their original sequence lengths.
        - Dropout is currently commented out but can be enabled for regularization.
    """
    def __init__(self, hidden_size=32, output_size=192, dropout=0.0, device=None):
        super().__init__()
        
        self.linear = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.device = device
        
    def forward(self, factors):
        # concatenate all sequences, forward into linear layer and split again
        lengths = [factor.shape[0] for factor in factors]
        cat_fs = torch.cat(factors, dim=0)
      #  cat_fs = self.dropout(cat_fs)
        rates = self.linear(cat_fs)
        rates = [rate for rate in torch.split(rates, lengths)]
        
        return rates

In [ ]:
class CoreAndReadout(nn.Module):
    """
    A PyTorch module that combines a core encoder and a readout decoder for sequential data processing.

    Args:
        core (nn.Module): The core module, typically an encoder, that processes the input data.
        readout (nn.Module): The readout module, typically a decoder, that generates the final output.

    Attributes:
        core (nn.Module): The core encoder module.
        readout (nn.Module): The readout decoder module.

    Methods:
        forward(x):
            Passes the input through the core module and then through the readout module.

    Forward Args:
        x (torch.Tensor): The input tensor to be processed by the core module.

    Returns:
        torch.Tensor: The output tensor after processing through the core and readout modules.

    Notes:
        - This class is designed to separate the encoding (core) and decoding (readout) stages of a model.
        - The `core` and `readout` modules can be any PyTorch `nn.Module` that fits the input/output requirements.
    """
    def __init__(self, core, readout):
        super().__init__()
        self.core = core
        self.readout = readout
        
    def forward(self, x):
        x = self.core(x)
        x = self.readout(x)
        return x

In [11]:
def get_device(cpu_preference=False, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() and not cpu_preference else "cpu")
    if verbose: print("Using device:", device)
    return device

In [12]:
def collate_packed_sequences(batch):
    """
    Collates a batch of sequences into a packed sequence for efficient processing in RNNs.

    Args:
        batch (list of tuples): A batch of data where each element is a tuple (X, Y).
            - X: A sequence (e.g., list or array) representing input features.
            - Y: A sequence (e.g., list or array) representing target labels.

    Returns:
        tuple:
            - PackedSequence: A packed representation of the input sequences (X) for use with PyTorch RNNs.
            - list: A list of target sequences (Y) converted to tensors and moved to the specified device.

    Notes:
        - The function converts input and target sequences to PyTorch tensors and moves them to the device.
        - The `pack_sequence` function is used to create a packed sequence from the input data, which is useful for variable-length sequences.
        - The `enforce_sorted=False` argument ensures that the sequences do not need to be sorted by length beforehand.
    """
    X, Y = zip(*batch)
    X = [torch.Tensor(elem).to(device) for elem in X]
    Y = [torch.Tensor(elem).to(device) for elem in Y]
    return pack_sequence(X, enforce_sorted=False), Y

In [ ]:
class TrialDataset_seq(Dataset):
    """
    A PyTorch Dataset for handling sequential trial-based data with optional moving average smoothing.

    Args:
        df (pd.DataFrame): A DataFrame containing the trial data.
        X (list of str): A list of column names representing the input features.
        y (list of str): A list of column names representing the target labels.
        seq_len (int): The size of the moving average window for smoothing. Default is 1 (no smoothing).

    Attributes:
        df (pd.DataFrame): The input DataFrame containing trial data.
        X (list of str): The input feature column names.
        y (list of str): The target label column names.
        bin_size (int): The size of the moving average window.
        trials (np.ndarray): An array of unique trial identifiers.

    Methods:
        __len__():
            Returns the number of unique trials in the dataset.

        __getitem__(idx):
            Retrieves the input features and target labels for a specific trial, applying moving average smoothing.

    Forward Args:
        idx (int): The index of the trial to retrieve.

    Returns:
        tuple:
            - torch.Tensor: A tensor of smoothed input features for the specified trial.
            - torch.Tensor: A tensor of smoothed target labels for the specified trial.

    Notes:
        - The moving average is applied to both input features and target labels using the specified `seq_len`.
        - The `rolling` method is used to compute the moving average, and rows with NaN values are dropped.
        - The dataset is designed to handle trial-based data, where each trial is treated as a separate sequence.
    """
    def __init__(self, df, X, y, seq_len=1):
        self.df = df
        self.X = X
        self.y = y
        self.bin_size = seq_len
        self.trials = df['trial'].unique()  # Unique trial identifiers
    
    def __len__(self):
        return len(self.trials)  # Number of trials
    
    def __getitem__(self, idx):
        # Get all rows for a specific trial
        trial_number = self.trials[idx]
        trial_data = self.df[self.df['trial'] == trial_number]
        
        # Compute the moving average for the features
        features = trial_data[self.X].rolling(window=self.bin_size).mean().dropna().values
        features_tensor = torch.tensor(features, dtype=torch.float32)
        
        # Compute the moving average for the labels (if needed)
        labels = trial_data[self.y].rolling(window=self.bin_size).mean().dropna().values
        labels_tensor = torch.tensor(labels, dtype=torch.float32)
        
        return features_tensor, labels_tensor

In [ ]:
class GRUDecoder(nn.Module):
    """
    A GRU-based decoder for processing sequential data and generating output predictions.

    Args:
        hidden_size (int): The size of the hidden state in the GRU layer. Default is 32.
        output_size (int): The size of the output predictions. Default is 192.
        num_layers (int): The number of stacked GRU layers. Default is 1.
        dropout (float): Dropout probability for regularization. Default is 0.0 (no dropout).
        device (torch.device, optional): The device to which the module is assigned. Default is None.

    Attributes:
        hidden_size (int): The size of the GRU hidden state.
        output_size (int): The size of the output predictions.
        num_layers (int): The number of GRU layers.
        device (torch.device): The device used for computations.
        gru (nn.GRU): A GRU layer for processing sequential data.
        fc (nn.Linear): A fully connected layer to map GRU outputs to the desired output size.
        dropout (nn.Dropout): A dropout layer for regularization.

    Methods:
        forward(factors):
            Processes a list of input sequences through the GRU and generates output predictions.

    Forward Args:
        factors (list of torch.Tensor): A list of input sequences, where each sequence is a tensor of shape 
            (sequence_length, hidden_size).

    Returns:
        list of torch.Tensor: A list of output sequences, where each sequence is a tensor of shape 
            (sequence_length, output_size).

    Notes:
        - The input `factors` must be a list of tensors, each representing a sequence.
        - The method concatenates all sequences, processes them through the GRU, and splits them back into their original lengths.
        - Dropout is applied to the GRU outputs before the linear transformation.
        - The GRU operates on sequences in a batch-first format, where the batch size is the total number of sequences.
    """
    def __init__(self, hidden_size=32, output_size=192, num_layers=1, dropout=0.0, device=None):
        super().__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.device = device

        # Define the GRU layer
        self.gru = nn.GRU(input_size=hidden_size, 
                          hidden_size=hidden_size, 
                          num_layers=num_layers, 
                          batch_first=True, 
                          dropout=dropout)

        # Output linear layer to transform GRU outputs to desired output size
        self.fc = nn.Linear(hidden_size, output_size)

        # Optional dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, factors):
        # Ensure the input is a list of sequences (tensors)
        if not isinstance(factors, list):
            raise ValueError("Expected input to be a list of sequences.")

        # Get the lengths of each sequence
        lengths = [factor.shape[0] for factor in factors]

        # Concatenate all sequences into a single tensor along the batch dimension
        # The batch size will be the total number of sequences, each with its own sequence length
        cat_fs = torch.cat(factors, dim=0).unsqueeze(1)  # Add batch dimension (for GRU)

        # Pass the concatenated sequence through the GRU
        gru_output, _ = self.gru(cat_fs)
        gru_output = self.dropout(gru_output)
        # Apply the linear transformation (fc) to the GRU output
        rates = self.fc(gru_output)

        # Remove the batch dimension and split the result back into the original list of sequences
        rates = rates.squeeze(1)  # Remove the added batch dimension

        # Split back into the original sequences based on the lengths
        rates = list(torch.split(rates, lengths))  # Convert tuple to list here

        # Return the list of output sequences
        return rates

In [ ]:
class ExperimentEncoder_LN(nn.Module):
    def __init__(self, input_size=12, hidden_size=32, num_layers=1, device=None):
        super().__init__()
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True, bias=False)
        
        # Initialize the hidden state for GRU
        self.initial_hidden_state = nn.Parameter(torch.zeros(1, 1, hidden_size))  # unidirectional GRU
        
        # Add layer normalization
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.device = device
        
    def forward(self, input_):
        # Input should be a PackedSequence
        if not isinstance(input_, PackedSequence):
            raise ValueError('Input must be a PackedSequence.')
        
        batch_size = input_.unsorted_indices.shape[0]  # Extract batch size from PackedSequence
        initial_hidden_state = self.initial_hidden_state.repeat(1, batch_size, 1)
        
        # Forward pass through GRU
        factors, _ = self.gru(input_, initial_hidden_state)
        
        # Return unpacked sequence
        factors = unpack_sequence(factors)
        factors[0] = self.layer_norm(factors[0])
        
        return factors

# Training Functions

In [ ]:
def create_lineplots(all_probs_array, y_val, y_labels, trial, shift, n_epochs, hidden_size, file_path, features, model_type, size=(18, 10), show_plot=True):
    # Create a figure with subplots for each variable
    fig, axs = plt.subplots(len(y_labels), 1, figsize=size,linewidth=0.01)
    
    if len(y_labels) == 1:
        axs = [axs]  # Wrap the single Axes object in a list   
    
    for i, label in enumerate(y_labels):
        ax = axs[i]
        ax.plot(y_val[:, i], label='True', linewidth=2.5, color='blue')  # True values in blue
        ax.plot(all_probs_array[:, i], label='Predicted', linewidth=2.5, color='orange')  # Predicted values in orange

        # Set titles and labels
        if i == 0:  # Only set title for the first plot
            ax.set_title(f'({model_type} (packing)) (val trial: {trial}, lag {shift}, epochs: {n_epochs}, hidden states: {hidden_size})')
            ax.legend()
        
        ax.set_xlabel('Time Step',labelpad=0.5)
        ax.set_ylabel(label)
        feature_names = ", ".join(features)
        fig.text(.5, .00001, f"Features: {feature_names}", ha='center', fontsize=14)
        
        if file_path is not None:
            plt.savefig(file_path)

    # Show the plots if requested
    if show_plot:
        plt.tight_layout()  # Adjust spacing between subplots for better layout
        plt.show()

In [ ]:
def train(dataloader, model, optimizer, loss_fn, device=None):
    epoch_loss = []
    epoch_correct, epoch_total = 0, 0
    
    model.train()
    for x, y in dataloader:
        optimizer.zero_grad()
        
        y_pred = model(*x)
        
        
        loss = loss_fn(y_pred, y)
        epoch_loss.append(loss.item())
        
        loss.backward()
        optimizer.step()
    
    return np.mean(epoch_loss), y_pred

In [ ]:
def validate(dataloader, model, loss_fn, device=None):
    epoch_loss = []
    epoch_correct, epoch_total = 0, 0
    
    model.eval()
    with torch.no_grad():
        for x, y in dataloader:
            y_pred = model(*x)
            loss = loss_fn(y_pred, y)
            epoch_loss.append(loss.item())
    
    return np.mean(epoch_loss), y_pred

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0, verbose=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
            min_delta (float): Minimum change in the monitored quantity to qualify as an improvement.
            verbose (bool): If True, prints messages about early stopping progress.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()  # Save the initial model state
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model_state = model.state_dict()  # Update the best model state
        else:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

    def restore_best_model(self, model):
        """
        Restores the model state to the best state observed during training.
        """
        if self.best_model_state is not None:
            model.load_state_dict(self.best_model_state)
        return model


In [ ]:
import torch

class M2S_Loss3(nn.Module):
    def __init__(self, criterion=None, pad_value=0.0):
        super().__init__()
        self.criterion = criterion if criterion is not None else nn.MSELoss(reduction='mean')
        self.pad_value = pad_value  # Define the padding value (default to 0.0)

    def forward(self, y_pred, y):
        if isinstance(y_pred, list) and isinstance(y, list):
            total_loss = 0
            total_valid_sequences = 0

            for i in range(len(y_pred)):
                pred_seq = y_pred[i]
                target_seq = y[i]

                # Ensure both sequences have the same length
                valid_length = min(pred_seq.shape[0], target_seq.shape[0])

                # Slice sequences to valid lengths (no padding assumed here)
                pred_seq = pred_seq[:valid_length]
                target_seq = target_seq[:valid_length]

                # Create a mask to ignore padded values (assuming padding is represented by pad_value)
                mask = (target_seq != self.pad_value).float()

                # Calculate the loss with the mask
                loss = self.criterion(pred_seq * mask, target_seq * mask)  # Apply mask to ignore padding
                total_loss += loss.sum()  # Sum loss over sequence
                total_valid_sequences += valid_length  # Count valid sequence length
            
            # Return the mean loss over all valid elements
            return total_loss / total_valid_sequences
        else:
            raise ValueError('y_pred and y should be lists of sequences.')


In [ ]:
def run_training_ES_C(
    train_dataloader, 
    val_dataloader, 
    model, 
    optimizer, 
    loss_fn, 
    num_epochs, 
    scheduler=None, 
    device=None, 
    verbose=False, 
    patience=20, 
    save_dir=".", 
    model_name="model"
    ):
    train_losses = []
    val_losses = []
    yPred_tr, yPred_val = [], []    

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=patience, verbose=verbose)

    for epoch in range(num_epochs):
        # Training step
        epoch_train_loss, yPred_tr = train(train_dataloader, model, optimizer, loss_fn, device)
        
        # Validation step
        epoch_val_loss, yPred_val = validate(val_dataloader, model, loss_fn, device)
        
        # Scheduler step
        if scheduler is not None:
            scheduler.step(epoch_train_loss)
        
        # Log the losses
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)

        # Early stopping check
        early_stopping(epoch_val_loss, model)

        # Save model every 200 epochs
        #if epoch % 200 == 0 and epoch != 0:
           # checkpoint_path = f'{save_dir}/{model_name}_epoch_{epoch}_checkpoint.pth'
            #torch.save(model.state_dict(), checkpoint_path)
            #print(f"Model checkpoint saved at epoch {epoch}: {checkpoint_path}")

        # Print training information
        if verbose or epoch % 50 == 0 and epoch != 0:
            print(f"Epoch {epoch}, train loss: {epoch_train_loss}, val loss: {epoch_val_loss}")
        
        # Break training if early stopping criteria are met
        if early_stopping.early_stop:
            print("Early stopping triggered. Restoring the best model state...")
            model = early_stopping.restore_best_model(model)
            break

    # Final restoration of the best model
    model = early_stopping.restore_best_model(model)


    # Save the final best model after all epochs
    best_model_path = f'{save_dir}/{model_name}_best_model.pth'
    torch.save(model.state_dict(), best_model_path)
    print(f"Final best model saved: {best_model_path}")
    return train_losses, val_losses, yPred_tr, yPred_val

In [ ]:
import pandas as pd

# Load your data
data = ''  # Path to your dataset
df = pd.read_csv(data)

# Filter trials (if necessary)
keep_trials = [1, 3, 5, 6, 10, 12, 13, 14, 15, 16, 20, 22, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40, 42, 43, 44, 45, 61]
if 'trial' in df.columns:
    df = df[df['trial'].isin(keep_trials)]

# Define fixed values for each scenario
scenarios = {
    1: {
        'oDistReach_1': 0.039192008, 'oDistReach_2': 0.159473814, 'oDistReach_3': 0.244949143, 'oDistReach_4': 0.40015473,
        'oAngHGFeed_1': 53.20439684, 'oAngHGFeed_2': 21.58116759, 'oAngHGFeed_3': 14.74049229, 'oAngHGFeed_4': 19.9144559
    },
    2: {
        'oDistReach_1': 0.168446136, 'oDistReach_2': 0.020607723, 'oDistReach_3': 0.161489225, 'oDistReach_4': 0.268309158,
        'oAngHGFeed_1': 29.60866461, 'oAngHGFeed_2': 19.4084781, 'oAngHGFeed_3': 59.38005131, 'oAngHGFeed_4': 68.25748778
    },
    3: {
        'oDistReach_1': 0.28621023,  'oDistReach_2': 0.168894108, 'oDistReach_3': 0.030250559, 'oDistReach_4': 0.146278534,
        'oAngHGFeed_1': 21.38281388, 'oAngHGFeed_2': 16.46802007, 'oAngHGFeed_3': 54.0270024,  'oAngHGFeed_4': 72.08987075
    },
    4: {
        'oDistReach_1': 0.420576716, 'oDistReach_2': 0.274890588, 'oDistReach_3': 0.153489252, 'oDistReach_4': 0.040491932,
        'oAngHGFeed_1': 41.21796036, 'oAngHGFeed_2': 17.32473936, 'oAngHGFeed_3': 23.17393409, 'oAngHGFeed_4': 43.38455563
    },
}

# Apply each scenario and save to a new file
for scenario, values in scenarios.items():
    df_scenario = df.copy()
    for column, value in values.items():
        df_scenario[column] = value

    # Save the scenario to a separate file
    output_file = f'dyadic_scenario_{scenario}_dataset.csv' # Output file name for the scenario
    df_scenario.to_csv(output_file, index=False)

print("Scenarios saved successfully!")

# Model Training

In [ ]:

from sklearn.preprocessing import MaxAbsScaler, RobustScaler, StandardScaler
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
lag_values = np.arange(1, 26)
folds = [[1, 16, 25, 30, 37]]
drop_trials = [4, 7, 21, 46, 47, 48, 54, 58, 17, 8, 52, 56, 17]
keep_trials = [1, 3, 5, 6, 10, 12, 13, 14, 15, 16, 20, 22, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40, 42, 43, 44, 45, 61]
test_trials = []
start = perf_counter()

feature_set1 = ["aAngHGFeed_1_lagged","aAngHGFeed_2_lagged","aAngHGFeed_3_lagged","aAngHGFeed_4_lagged"]
feature_set2 = ["aAngHG_oHead_lagged","aAngHG_oMid_lagged","aAngHG_oReach_lagged"]
feature_set3 = ["aDistReach_1_lagged","aDistReach_2_lagged","aDistReach_3_lagged","aDistReach_4_lagged"]
feature_set4 = ["oDistReach_1_lagged","oDistReach_2_lagged","oDistReach_3_lagged","oDistReach_4_lagged"]
feature_set5 = ["oAngHGFeed_1_lagged","oAngHGFeed_2_lagged","oAngHGFeed_3_lagged","oAngHGFeed_4_lagged"]
standard_scaler = MinMaxScaler()
feature_sets = [feature_set1, feature_set2, feature_set3, feature_set4, feature_set5]
hidden_values = [24,32,64,128]
lag_values = np.arange(1,26)
# Iterate over different lag values
for h in hidden_values:
    for s in lag_values:
        # Define the feature sets based on model type
        model_type = 'full'
        hidden_size = h
        set_value = [1, 16, 25, 30, 37]
        dec = 'gd'
        n_epochs = 2000
        shift = s
        model_name = f'{model_type}_h{hidden_size}_e{n_epochs}_v{set_value}_s{shift}'
        labels = ['aDistReach_1', 'aDistReach_2', 'aDistReach_3', 'aDistReach_4']
        save_dir = f'/kaggle/working/{model_type}_h{hidden_size}'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
    
        # Prepare data file and load the dataset
        data = '/kaggle/input/df-filtered-bin/df_binned.csv'
    
        # Columns for shifting
        columns_to_shift = ['aAngHGFeed_1', 'aAngHGFeed_2', 'aAngHGFeed_3', 'aAngHGFeed_4',
       'aAngHG_oHead', 'aAngHG_oMid', 'aAngHG_oReach', 'aDistReach_1',
       'aDistReach_2', 'aDistReach_3', 'aDistReach_4', 'oDistReach_1',
       'oDistReach_2', 'oDistReach_3', 'oDistReach_4', 'oAngHGFeed_1',
       'oAngHGFeed_2', 'oAngHGFeed_3', 'oAngHGFeed_4']
        feature_sets = [feature_set1,feature_set2,feature_set3,feature_set4,feature_set5]
        df = pd.read_csv(data)
        df = df[df['trial'].isin(keep_trials)]
        
    
        # Apply log transformation to the distance variables for better prediction near zero
        df[['oDistReach_1', 'oDistReach_2', 'oDistReach_3', 'oDistReach_4']] = np.log(df[['oDistReach_1', 'oDistReach_2', 'oDistReach_3', 'oDistReach_4']] + 1e-5)
        df[['aDistReach_1', 'aDistReach_2', 'aDistReach_3', 'aDistReach_4']] = np.log(df[['aDistReach_1', 'aDistReach_2', 'aDistReach_3', 'aDistReach_4']] + 1e-5)
        # Handle angle variables (scale them differently)
    
        angle_columns = ["oAngHGFeed_1","oAngHGFeed_2","oAngHGFeed_3","oAngHGFeed_4"]
        df[angle_columns] = MinMaxScaler().fit_transform(df[angle_columns])
    
            # Scale angle variables using MaxAbsScaler for proportional consistency
            
        lag_word = '_lagged'
        input_size = len(columns_to_shift)
        output_size = len(labels)
        # Now create lagged columns
        shifted_df = pd.DataFrame()
        for trial_value in df['trial'].unique():
            trial_df = df[df['trial'] == trial_value].copy()
            for col in columns_to_shift:
                new_col_name = col + '_lagged'
                trial_df[new_col_name] = trial_df[col].shift(-s)
            trial_df = trial_df.dropna()
            shifted_df = pd.concat([shifted_df, trial_df], ignore_index=True)
    
        # Train and validation set preparation
        features = shifted_df.columns[shifted_df.columns.str.contains('_lagged')]
        train_set = shifted_df[~shifted_df['trial'].isin(set_value)]
        train_dataset = TrialDataset_seq(train_set, feature_sets, y=labels, seq_len=1)
        train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True, collate_fn=collate_packed_sequences)
    
        val_set = shifted_df[shifted_df['trial'].isin(set_value)]
        val_dataset = TrialDataset_seq(val_set, feature_sets, y=labels, seq_len=1)
        val_dataloader = DataLoader(val_dataset, batch_size=len(set_value), shuffle=False, collate_fn=collate_packed_sequences)
    
                # Initialize encoders for each feature set
        encoders = [ExperimentEncoder_LN(input_size=len(feature_set), hidden_size=hidden_size) for feature_set in feature_sets]
        combined_encoder = CombinedEncoder(encoders)
        if dec == 'ld':
            readout = NonlinearDecoder(hidden_size * len(feature_sets), output_size)
        if dec == 'gd':
            readout = GRUDecoder(input_size=hidden_size * len(feature_sets), hidden_size=hidden_size, output_size=output_size)
        model = CoreAndReadout(combined_encoder, readout)
        model = model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, threshold=0.0001)
        loss = nn.MSELoss()
        loss_fn = M2S_Loss3(loss)
    
    
        models_dir = save_dir + "/models/"
        if not os.path.exists(models_dir):
            os.makedirs(models_dir)
        train_losses, val_losses, yPred_tr, yPred_val = run_training_ES_C(train_dataloader, val_dataloader, model, optimizer, loss_fn,
                                                                          n_epochs, scheduler=scheduler, device=device, verbose=False, patience=50, save_dir=models_dir, model_name=model_name)
    
        fig, ax = plt.subplots(figsize=(18, 4))
        ax.plot((train_losses), label='Train Loss', color='blue')
        ax.plot((val_losses), label='Validation Loss', color='orange')
        ax.set_title((f'Learning curve ({model_type} - packed): (val trial: {set_value}, lag {shift}, epochs: {n_epochs}, hidden states: {hidden_size})'))
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.legend(loc='upper right')
        ax.grid()
        plot_name = f'{model_type}_{set_value}_e{n_epochs}_h{hidden_size}_l{shift}_LC.eps'
        plt.savefig(plot_name, format='eps')
        lc_folder =  save_dir + "/LC/"
        if not os.path.exists(lc_folder):
            os.makedirs(lc_folder)
        lc_dir = os.path.join(lc_folder, plot_name)
        plt.savefig(lc_dir)
        plt.show()
    
        model.eval()
    
    
        with torch.no_grad():
            for batch_idx, (inputs, outputs) in enumerate(val_dataloader):
                inputs = [input_.to(device) for input_ in inputs]
                outputs = [output.to(device) for output in outputs]
                y_pred_batch = model(*inputs)
        
                for i in np.arange(len(y_pred_batch)):
                    print(f'Trial: {set_value[i]}')
        
                    prediction = y_pred_batch[i]
                    true_value = outputs[i]
        
                    # Ensure both prediction and true_value have the same length
                    valid_length = min(prediction.shape[0], true_value.shape[0])
                    prediction = prediction[:valid_length]  # Cut prediction to valid length
                    true_value = true_value[:valid_length]  # Cut true_value to valid length
        
                    # Convert to numpy for plotting
                    prediction_np = prediction.detach().cpu().numpy()
                    true_value_np = true_value.detach().cpu().numpy()
        
                    # Save the plot with a descriptive filename
                    plot_filename = f'{model_type}_v{set_value[i]}_e{n_epochs}_h{hidden_size}_l{shift}.png'
                    plot_dir = os.path.join(save_dir, plot_filename)
                    # Call the plot function with the adjusted predictions and true values
                    create_lineplots(
                        all_probs_array=np.exp(prediction_np),  # Assuming you want to plot the exp of values
                        y_val=np.exp(true_value_np),  # Apply exp to the true values as well
                        y_labels=labels,
                        trial=set_value[i],
                        shift=shift,
                        n_epochs=n_epochs,
                        hidden_size=hidden_size,
                        file_path=None,
                        features=features,
                        size=(20, 12),
                        show_plot=True,
                        model_type=model_type
                    )


end = perf_counter()
print(f"Time taken to execute code : {end-start}")

# Evaluation and saving results

In [ ]:
# dropping trials shorter than 3.5 sec and these with too many missing values
drop_trials = [4, 7, 21, 46, 47, 48, 54, 58, 17, 8, 52, 56, 17]
keep_trials = [1, 3, 5, 6, 10, 12, 13, 14, 15, 16, 20, 22, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40, 42, 43, 44, 45, 61]
len(keep_trials)

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
angle_scaler = MinMaxScaler()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
drop_trials = [4, 7, 21, 46, 47, 48, 54, 58, 17, 8, 52, 56, 17]
keep_trials = [1, 3, 5, 6, 10, 12, 13, 14, 15, 16, 20, 22, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40, 42, 43, 44, 45, 61]
folds = [1, 16, 25, 30, 37]
models = ['solo','dyadic','full']
lag_values = np.arange(1, 26)
standard_scaler = MinMaxScaler()
decoders = ['gd','ld']


# Initialize a list to store ALL results outside all loops
all_results = []
hidden_values = [16]
for d in decoders:
    for m in models:
        for shift in lag_values:
            for val in folds:
                # Set up parameters
                dec = d
                model_type = m
                hidden_size = 16
                n_epochs = 2000
                model_name = f'{model_type}_h{hidden_size}_e{n_epochs}_v[1, 16, 25, 30, 37]_s{shift}'
                model_path = f'/{model_type}_h{hidden_size}_{dec}_1enc/models/{model_name}_best_model.pth'
                predictions_dir = f'/{model_type}_h{hidden_size}_{dec}_1enc/predictions'
                data = f'datasets/df_binned_filtered.csv'

                # Define labels and columns
                labels = ['aDistReach_1', 'aDistReach_2', 'aDistReach_3', 'aDistReach_4']
                if model_type == 'full':
                    columns_to_shift = ['aAngHGFeed_1', 'aAngHGFeed_2', 'aAngHGFeed_3', 'aAngHGFeed_4',
                                    'aAngHG_oHead', 'aAngHG_oMid', 'aAngHG_oReach', 'aDistReach_1',
                                    'aDistReach_2', 'aDistReach_3', 'aDistReach_4', 'oDistReach_1',
                                    'oDistReach_2', 'oDistReach_3', 'oDistReach_4', 'oAngHGFeed_1',
                                    'oAngHGFeed_2', 'oAngHGFeed_3', 'oAngHGFeed_4']
                elif model_type == 'dyadic':
                    columns_to_shift = ['aDistReach_1', 'aDistReach_2', 'aDistReach_3', 'aDistReach_4', 'oDistReach_1',
                                    'oDistReach_2', 'oDistReach_3', 'oDistReach_4']
                else:
                    columns_to_shift = ['aDistReach_1', 'aDistReach_2', 'aDistReach_3', 'aDistReach_4']

                # Load and prepare data
                lag_word = '_lagged'
                input_size = len(columns_to_shift)
                output_size = len(labels)
                df = pd.read_csv(data)
                df = df[df['trial'].isin(keep_trials)]
                if model_type == 'full' or model_type == 'dyadic':
                    df[['oDistReach_1', 'oDistReach_2', 'oDistReach_3', 'oDistReach_4']] = np.log(df[['oDistReach_1', 'oDistReach_2', 'oDistReach_3', 'oDistReach_4']] + 1e-5)
                df[['aDistReach_1', 'aDistReach_2', 'aDistReach_3', 'aDistReach_4']] = np.log(df[['aDistReach_1', 'aDistReach_2', 'aDistReach_3', 'aDistReach_4']] + 1e-5)

                # Handle angle variables
                if model_type == 'full':
                    angle_columns = ['aAngHGFeed_1', 'aAngHGFeed_2', 'aAngHGFeed_3', 'aAngHGFeed_4',
                                'aAngHG_oHead', 'aAngHG_oMid', 'aAngHG_oReach',
                                'oAngHGFeed_1', 'oAngHGFeed_2', 'oAngHGFeed_3', 'oAngHGFeed_4']
                    df[angle_columns] = angle_scaler.fit_transform(df[angle_columns])

                # Apply lag shifting
                shifted_df = pd.DataFrame()
                for trial_value in df['trial'].unique():
                    trial_df = df[df['trial'] == trial_value].copy()
                    for col in columns_to_shift:
                        trial_df[f'{col}{lag_word}'] = trial_df[col].shift(-shift)
                    trial_df = trial_df.dropna()
                    shifted_df = pd.concat([shifted_df, trial_df], ignore_index=True)

                # Extract features and prepare datasets
                features = shifted_df.columns[shifted_df.columns.str.contains(lag_word)]
                val_set = shifted_df[shifted_df['trial'] == val]
                
                val_dataset = TrialDataset_seq(val_set, X=features, y=labels, seq_len=1)
                val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_packed_sequences)

                # Load model
                core = ExperimentEncoder_LN(input_size, hidden_size, num_layers=1)
                if dec == 'ld':
                    readout = NonlinearDecoder(hidden_size, output_size)
                if dec == 'gd':
                    readout = GRUDecoder(hidden_size,output_size)
                model = CoreAndReadout(core, readout)
                model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
                model = model.to(device)
                model.eval()

                # Define loss functions
                loss_fn = M2S_Loss3(nn.MSELoss())

                # Evaluate
                with torch.no_grad():
                    for batch_idx, (inputs, outputs) in enumerate(val_dataloader):
                        inputs = inputs.to(device)
                        outputs = [output.to(device) for output in outputs]
                        y_pred_batch = model(inputs)

                        for i in range(len(y_pred_batch)):
                            prediction = y_pred_batch[i]
                            prediction_np = np.exp(prediction.detach().cpu().numpy())
                            true_value_np = np.exp(outputs[i].detach().cpu().numpy())

                            r2_h = r2_score(true_value_np, prediction_np)
                            mse_h = np.mean((true_value_np - prediction_np) ** 2)
                            ade_h = np.mean(np.linalg.norm(prediction_np - true_value_np, axis=1))
                            mape_h = np.mean(np.abs((true_value_np - prediction_np) / true_value_np)) * 100
                            rmse_h = np.sqrt(mse_h)

                            # Store results with decoder type
                            all_results.append({
                                'model_type':  model_type + "_" + dec + "_1enc",
                                'shift': shift,
                                'val': val,
                                'hidden_size': hidden_size,
                                'r2': r2_h,
                                'mse': mse_h,
                                'ade': ade_h,
                                'mape': mape_h,
                                'rmse': rmse_h
                            })

                            # Save predictions to file
                            prediction_df = pd.DataFrame({
                                **{f'true_{label}': true_value_np[:, idx] for idx, label in enumerate(labels)},
                                **{f'pred_{label}': prediction_np[:, idx] for idx, label in enumerate(labels)}
                            })
                            prediction_file_path = f'{predictions_dir}/trial{val}_shift{shift}_hidden{hidden_size}.csv'
                            if not os.path.exists(predictions_dir):
                                os.makedirs(predictions_dir)
                            prediction_df.to_csv(prediction_file_path, index=False)

        print(f"Completed evaluating {model_type} model with {dec} decoder")

# Save all results to a single CSV after all evaluations are complete
final_results_df = pd.DataFrame(all_results)
final_results_path = 'results.csv'
final_results_df.to_csv(final_results_path, index=False)
print(f"Evaluation complete. All results saved to {final_results_path}")